
## "NL2SQLAgent": WXAI VERSION


- this version is meant to be executed in WATSONX_AI

## Install Dependencies and restart kernel

In [ ]:
!pip install pydantic==2.11.1 langchain==0.3.22 langchain-core==0.3.51 langchain-ibm==0.3.10 langchain-text-splitters==0.3.8 PyHive==0.7.0

In [ ]:
# Restart kernel
import os
os._exit(00)

In [ ]:
#test for potential failure on this package
from langgraph.prebuilt import create_react_agent

## Import libraries

In [ ]:
import os
from ibm_watsonx_ai import Credentials, APIClient
from ibm_watsonx_ai.deployments import RuntimeContext
import getpass

from dotenv import load_dotenv

## Load connections information from the project

In [ ]:
# list your connections
display(wslib.list_connections())
# make sure you use the right connection name for presto
presto_conn = wslib.get_connection('presto_connection')
cos_conn = wslib.get_connection('cos_connection')

### Load env.txt file with configurations

In [ ]:
with open('.env_all', 'wb') as env_file:
    env_file.write(wslib.load_data('env.txt').read())
# environmental variables store credentials and configuration
load_dotenv('.env_all')

### Enter your Cloud API Key

In [ ]:
CLOUD_API_KEY = getpass.getpass("Enter your Cloud API key: ")
print("Cloud API Key recieved")

In [ ]:
# settings
params = {
        "space_id": os.getenv("WATSONX_DEPLOYMENT_SPACE_ID"),
        "project_id" : os.getenv("WATSONX_PROJECT_ID"),
        "api_key": CLOUD_API_KEY,
        "presto_host" : presto_conn["engine_host"],
        "presto_port" : presto_conn["engine_port"],
        "presto_user" : "ibmlhapikey",
        "presto_key" : presto_conn["api_key"],
        "clients_schema": os.getenv("SCHEMA_DATA_I"),
        "service_url" : os.getenv("WATSONX_URL")
    }


assert params['api_key'], "could not load environment variables properly"

display("Check you parameters", params)

In [ ]:
credentials = Credentials(
    url = params['service_url'],
    api_key = params['api_key']    
)

space_id = params['space_id'] 
project_id = params['project_id'] 
api_client = APIClient(credentials=credentials)

## Deployable service

In [ ]:
def gen_ai_service(context, params = params, **custom):    

    from ibm_watsonx_ai import APIClient, Credentials    
    from langchain_ibm import WatsonxLLM    
    from langchain_core.tools import tool        
    from langchain.prompts import PromptTemplate
    from langchain_core.output_parsers import JsonOutputParser    
    from langchain_core.messages import AIMessage, HumanMessage
    from pydantic import BaseModel, Field        
    from langchain_ibm import ChatWatsonx
    from pyhive import presto
    from pyhive.exc import DatabaseError    
    from langgraph.prebuilt import create_react_agent
        
    service_url = params.get("service_url")
    space_id = params.get("space_id")
    project_id = params.get("project_id")
    api_key = params.get("api_key")
    presto_host = params.get("presto_host")
    presto_port = params.get("presto_port")
    presto_user = params.get("presto_user")
    presto_key = params.get("presto_key")
    clients_schema = params.get("clients_schema")

    api_client = APIClient(
        credentials=Credentials(url=service_url, token=context.generate_token()),
        space_id= space_id
    )
    model_id="meta-llama/llama-3-3-70b-instruct"
    
    api_client.set.default_space(space_id)


    def gen_ai_service_with_langgraph():

        def create_llm(max_new_tokens=1000, min_new_tokens=100, temperature=0, decoding_method='greedy', repetition_penalty=1, model_id="meta-llama/llama-3-3-70b-instruct"):
            parameters = {
                "decoding_method": decoding_method,
                "max_new_tokens": max_new_tokens,
                "min_new_tokens": min_new_tokens,
                "repetition_penalty": repetition_penalty                
            }            
            return WatsonxLLM(
                model_id= model_id,
                url=service_url,
                apikey=api_key,
                project_id=project_id,
                params=parameters,
            )

        model = create_llm()
        chat = ChatWatsonx(model_id=model_id, watsonx_client=api_client)
        class sql_code(BaseModel):
            """Schema for Python solutions."""
            prefix: str = Field(description="Description of the problem and approach")
            code: str = Field(description="Code block not including import statements")

        # Tool creation.
        @tool(parse_docstring=True)
        def text2SQL(query: str) -> str:
            """
            text2SQL tool which generates SQL code and executes it against the database for a given natural language query\
                about a customer's personal information such as name, address, profession, zip code as well as their accounts and stock holdings status.

            Args:
                query: The natural language query. customer_id MUST be used when searching for information related to a specific client

            Returns:
                The result from the database including the primary key used for further queries
            """

            print("-------------Generating SQL-------------------")
            print("User query:",query)            
            max_retries = 5

            def text2SQL_chain():
                prompt = PromptTemplate(
                    template="""
                    <|begin_of_text|><start_header_id|>system<|end_header_id|>
                    You are an expert in SQL and understanding the relationship between tables. You are to generate executable SQL queries specifically for PrestoDB that answers the user's questions provided in English.
                    The table schema metadata is also provided to assist you in generating these queries.
                    Always use the following instructions to format your response to user queries:\n {format_instructions}
                    <start_header_id|>user<|end_header_id|>

                    The instructions are as follows:
                    1. Understand the user's question and pull together all relevant tables as necessary.
                    2. Understand the schema of the tables and how they will be joined together.
                    3. Provide the output in the json format provided with executable SQL queries to get the desired results.
                    4. Ensure the column you are querying corresponds to the correct table and column name.
                    5. ALWAYS return ALL the information from the customers_table in your queries to ensure follow up questions can be answered easily.
                    6. DO NOT return any additional text other than the JSON object.

                    Table Schema: 
                    There are 3 tables of interest.


                    Table 1: postgres_catalog.bankdemo.customers_table containing the following columns: customer_id (primary key), name, address, zip_code, credit_rating, age,\
                        gender, marital_status, profession, nbr_years_cli, risk_score, city, state, and profile_url
                    Table 2: iceberg_data.{clients_schema}.accounts_table containing the following columns: customer_id (foreign key), account_id (primary key)
                    Table 3: iceberg_data.{clients_schema}.holdings_table containing the following columns: account_id (foreign key), holding_id (primary key), asset_ticker, holding_amt and tax_liability

                    Example queries:1. How many rows are there in customers table.
                            Answer: SELECT COUNT(*) as row_count FROM postgres_catalog.bankdemo.customers_table

                    Question: {question}<|eot_id|>
                    <start_header_id|>assistant<|end_header_id|>

                    JSON Output:
                    """,
                    input_variables=["question", "format_instruction", "clients_schema"]
                )
                t2s_chain = prompt | model
                return t2s_chain
            chain = text2SQL_chain()
            parser = JsonOutputParser(pydantic_object=sql_code)
            format_instructions = parser.get_format_instructions()
            result = chain.invoke({"question": query, "format_instructions": format_instructions, "clients_schema": clients_schema})
            print("-----------------------FINISHED GENERATING----------------------")
            count = 1
            qry = ""
            conn = presto.connect(host=presto_host, port=presto_port, username=presto_user, password=presto_key, protocol="https")
            response = ""
            print(result)
            while count <= max_retries:
                try:
                    parsed_result = parser.invoke(result)
                    qry = parsed_result["code"].replace(';', '')
                    print("------------------QUERY SQL----------------------")
                    print(qry)
                    cursor = conn.cursor()
                    cursor.execute(qry)
                    response = cursor.fetchall()            
                    print("-------------------SQL Execution Success!---------------------")
                    print("response:",response)
                    break
                except DatabaseError as e:
                    print(f"--------------------Error with SQL...Regenerating--------------------{count} out of 5 times")
                    response = ""
                    cursor.close()
                    result = chain.invoke({"question": f"The original query was {query} but your last generation of {result} returned database error {e}. Please fix your response and regenerate", "format_instructions": format_instructions, "clients_schema": clients_schema})
                    print(f"Error was {e}")
                    count += 1
                except Exception as e:
                    print(f"--------------------Error with Parsing SQL...Regenerating--------------------{count} out of 5 times")
                    response = ""
                    result = chain.invoke({"question": f"The original query was {query} but your last generation of {result} returned parsing error {e}. Please fix your response and regenerate.", "format_instructions": format_instructions, "clients_schema": clients_schema})
                    print(f"Error was {e}")
                    # print(result)
                    count += 1

            if not response:
                print("Failed to generate SQL from given text or no results were returned")
                return "No results found"

            return response
        
        tools = [text2SQL]

        # system_prompt = """You are a helpful AI assistant, please respond to the user's query to the best of your ability!
        # Take advantage of the tools and DO NOT generate any SQL yourself. Call the tool with natural language when information from the database is needed.        
        # When calling the SQL generation tool, use the customer_id when it is available.
        # You MUST display all information the tool output in a table format when it makes sense.
        # The database contains the following information and can be queried using the text2SQL tool: customer_id, name, address, zip code, profession, stocks holdings, holdings amount, city, state, and risk score.        
        # """
        
        langgraph_agent_executor = create_react_agent(chat, tools)
                        
        return langgraph_agent_executor
    
    def convert_messages(messages):
        converted_messages = []
        for message in messages:
            if (message["role"] == "user"):
                converted_messages.append(HumanMessage(content=message["content"]))
            elif (message["role"] == "assistant"):
                converted_messages.append(AIMessage(content=message["content"]))
        return converted_messages
        
    def generate(context) -> dict:
        
        api_client.set_token(context.get_token())   
        payload = context.get_json()     
        messages = payload["messages"]                
        graph = gen_ai_service_with_langgraph()        
        print(convert_messages(messages))
        response = graph.invoke({
                                    "messages": convert_messages(messages)
                                 })        
        last_message = response["messages"][-1]
        generated_response = last_message.content
        execute_response = {
            "headers": {
                "Content-Type": "application/json"
            },
            "body": {
                "choices": [{
                    "index": 0,
                    "message": {
                       "role": "assistant",
                       "content": generated_response
                    }
                }]
            }
        }                
        return execute_response    
    
    def generate_stream(context):
        api_client.set_token(context.get_token())
   
        payload = context.get_json()     
        messages = payload["messages"]                
        graph = gen_ai_service_with_langgraph()        
        print(convert_messages(messages))
        response = graph.invoke({
                                    "messages": convert_messages(messages)
                                 })        
        last_message = response["messages"][-1]
        generated_response = last_message.content        
        for chunk in generated_response:
            chunk_response = {
                "choices": [{
                    "index": 0,
                    "delta": {
                        "role": "assistant",
                        "content": chunk
                    }
                    
                }]
            }
            yield chunk_response
                    
    return generate, generate_stream

## Test Locally

In [ ]:
context = RuntimeContext(api_client=api_client)

streaming = True
findex = 1 if streaming else 0

local_function = gen_ai_service(context, space_id=space_id)[findex]

question= "Who are the clients (including names and IDs) who invested in IBM and is holding more than 10000. Give me just 5 such clients"        
# question = "How many rows are there in customers_table "

messages =[{ "role" : "user", "content": question }]

context.request_payload_json = {"messages":messages}

response = local_function(context)

if (streaming):
    print(response)
    for chunk in response:
        print(chunk["choices"][0]["delta"]["content"], end="", flush=True)
else:
    print(response)

response

# Deploy Function

### Configure the yaml file

In [ ]:
api_client.set.default_space(space_id)

In [ ]:
config_yml =\
"""
name: python311
channels:
  - empty
dependencies:
  - pip:
    - langchain-ibm==0.3.5    
    - langchain==0.3.22
    - langchain-community==0.3.20
    - langchain-core==0.3.50
    - langgraph==0.2.70          
    - pydantic==2.10.6
    - PyHive==0.7.0
    
prefix: /opt/anaconda3/envs/python311
"""

with open("config_nl2sql.yaml", "w", encoding="utf-8") as f:
    f.write(config_yml)

## Register and Store Environment with Package Extension Metadata

In [ ]:
from datetime import datetime
deployment_name = 'nl2sql-wxdata-deploy-' + datetime.now().strftime("%Y%m%d_%H%M%S")

In [ ]:
base_sw_spec_id = api_client.software_specifications.get_id_by_name("runtime-24.1-py3.11")
meta_prop_pkg_extn = {
    api_client.package_extensions.ConfigurationMetaNames.NAME: deployment_name,
    api_client.package_extensions.ConfigurationMetaNames.DESCRIPTION: "Environment with langchain ai",
    api_client.package_extensions.ConfigurationMetaNames.TYPE: "conda_yml"
}

pkg_extn_details = api_client.package_extensions.store(meta_props=meta_prop_pkg_extn, file_path="config_nl2sql.yaml")
pkg_extn_id = api_client.package_extensions.get_id(pkg_extn_details)
pkg_extn_id

## Create and Store Software Specification with Package Extension

In [ ]:
meta_prop_sw_spec = {
    api_client.software_specifications.ConfigurationMetaNames.NAME: deployment_name,
    api_client.software_specifications.ConfigurationMetaNames.DESCRIPTION: "Software specification deployment",
    api_client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {"guid": base_sw_spec_id}
}

sw_spec_details = api_client.software_specifications.store(meta_props=meta_prop_sw_spec)
sw_spec_id = api_client.software_specifications.get_id(sw_spec_details)
api_client.software_specifications.add_package_extension(sw_spec_id, pkg_extn_id)
sw_spec_id

## Store AI Service with Software Specification

In [ ]:

meta_props = {
    api_client.repository.AIServiceMetaNames.NAME: f"AI service {deployment_name}",
    api_client.repository.AIServiceMetaNames.SOFTWARE_SPEC_ID: sw_spec_id
}
stored_ai_service_details = api_client.repository.store_ai_service(gen_ai_service, meta_props)


## Retrieve Stored AI Service ID 

In [ ]:
ai_service_id = api_client.repository.get_ai_service_id(stored_ai_service_details)
ai_service_id

## Deploy AI Service with Crew AI Agent

In [ ]:
meta_props = {
    api_client.deployments.ConfigurationMetaNames.NAME: deployment_name,
    api_client.deployments.ConfigurationMetaNames.ONLINE: {},
}

deployment_details = api_client.deployments.create(ai_service_id, meta_props)

In [ ]:
deployment_id = api_client.deployments.get_id(deployment_details)
deployment_id

# Test Deployment

In [ ]:
{"messages":
 [
     { "role" : "user", 
      "content": "Who are the clients (including names and IDs) who invested in IBM and is holding more than 10000. Give me just 5 such clients" 
      }
      ]
      }

In [ ]:
# history="Who are the clients (including names and IDs) who invested in IBM and is holding more than 10000. Give me just 5 such clients"
question= "Who are the clients (including names and IDs) who invested in IBM and is holding more than 10000. Give me just 5 such clients"        

messages =[{ "role" : "user", "content": question }]

    
request_payload_json = {"messages":messages}

deployments_results = api_client.deployments.run_ai_service(
    deployment_id, request_payload_json #{"history": history}
)
deployments_results